# Лабораторная работа №4. Классы.
Общее задание

Необходимо переделать лабораторную работу №3 с использованием классов, описывающих предметную область, заданную вариантом, с реализацией следующих особенностей (вполне возможно, что предлагаемое в 3 лабе задание для этого нужно будет расширить):

    Класс должен содержать итератор
    Должна быть реализована перегрузка стандартных операций (repr, например)
    Должно быть реализовано наследование
    Запись значений в свойства - только через setattr
    Возможность доступа к элементам коллекции по индексу (getitem)
    Должны быть реализованы статические методы
    Должны быть реализованы генераторы


In [2]:
import os
import pandas as pd

class CSVDataHandler:
    def __init__(self, filename='Lr3.csv'):
        self.path = os.getcwd()
        self.filename = filename
        self.data = self._read_data()
        self.current_index = 0
        
        # Отключение стандартного столбца нумерации
        pd.set_option('display.show_dimensions', False)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', None)
    
    def _read_data(self): # Чтение данных из CSV файла
        file_path = os.path.join(self.path, self.filename)
        return pd.read_csv(file_path, sep='\t')
    
    def __iter__(self): # Инициализация итератора
        self.current_index = 0
        return self
    
    def __next__(self): # Получение следующей строки данных
        
        if self.current_index < len(self.data):
            row = self.data.iloc[self.current_index]
            self.current_index += 1
            return row
        else:
            raise StopIteration
    
    def __getitem__(self, index): # Доступ к элементам по индексу
        return self.data.iloc[index]
    
    def generator(self): # Генератор для последовательного доступа к строкам      
        for i in range(len(self.data)):
            yield self.data.iloc[i]

    def display(self, rows_per_page=50): # Постраничный вывод данных
        total_rows = len(self.data)
        for i in range(0, total_rows, rows_per_page):
            print(f"\nСтроки {i+1}-{min(i+rows_per_page, total_rows)} из {total_rows}:")
            with pd.option_context('display.max_rows', rows_per_page):
                print(self.data.iloc[i:i+rows_per_page])
            if i + rows_per_page < total_rows:
                input("Нажмите Enter для продолжения...")
    
    def __len__(self): # Количество строк данных
        return len(self.data)

    def __str__(self):
        return f"CSVDataHandler with {len(self)} rows"
            
    def add_row_from_console(self): # Добавление новой строки через консольный ввод
        
        print("\nДобавление новой строки:")
        print("Текущие столбцы:", list(self.data.columns))
        
        new_row = {}
        for column in self.data.columns:
            value = input(f"Введите значение для столбца '{column}': ")
            
            # Преобразование типов
            try:
                if pd.api.types.is_numeric_dtype(self.data[column]):
                    value = float(value) if '.' in value else int(value)
            except ValueError:
                pass
                
            new_row[column] = [value]
        
        # Добавление новой строки
        new_df = pd.DataFrame(new_row)
        self.data = pd.concat([self.data, new_df], ignore_index=True)
        print("\nСтрока успешно добавлена!")
        self.display()
        
    def save_to_csv(self): #Сохранение изменений в исходный файл
        file_path = os.path.join(self.path, self.filename)
        self.data.to_csv(file_path, sep='\t', index=False)
        print(f"\nДанные сохранены в файл: {file_path}")
        
    def delete_row_by_index(self, index):
        """
        Удаление строки по индексу
        :param index: Индекс строки для удаления
        :return: True если удаление успешно, False если индекс неверный
        """
        if 0 <= index < len(self.data):
            self.data = self.data.drop(index).reset_index(drop=True)
            print(f"Строка с индексом {index} успешно удалена!")
            self.display()
            return True
        else:
            print(f"Ошибка: Некорректный индекс {index}. Допустимые значения: 0-{len(self.data)-1}")
            return False
    



if __name__ == "__main__":
    handler = CSVDataHandler()
    while True:
        print("\nМеню:")
        print("1 - Вывести всю таблицу")
        print("2 - Вывести строку по индексу")
        print("3 - Вывести строки итератором")
        print("4 - Вывести строки генератором")
        print("5 - Добавить строку")
        print("6 - Удалить строку по индексу")
        print("7 - Сохранить изменения") 
        print("0 - Выход")

        choice = input("Выберите действие: ")
        match choice:
            case '1': # Вывести всю таблицу
                handler.display()
                
            case '2': # Вывести строку по индексу
                i = int(input("Введите номер строки: "))
                print(handler[i])
                    
            case '3': # Использование итератора
                print("\nИтерация по первым 3 строкам:")
                for i, row in enumerate(handler):
                    if i >= 3:
                        break
                    print(row)
                    print("-----")
                    
            case '4': # Использование генератора
                print("\nГенератор для строк:")
                gen = handler.generator()
                n = int(input("Строка начальная"))
                m = int(input("Строка конечная"))
                for i, row in enumerate(gen):
                    if i < n:  
                        continue
                    if i >= m:
                        break
                    print(row)
                    print("-----")

            case '5': # Ввод новой строки из консоли
                handler.add_row_from_console()
                
            case '6': # Удаление строки
                try:
                    index = int(input("Введите индекс строки для удаления: "))
                    handler.delete_row_by_index(index)
                except ValueError:
                    print("Ошибка: Введите целое число!")
                    
            case '7': # Сохранение в файл
                handler.save_to_csv()
            case '0': # Выход
                break
            case _:
                print("Неверный ввод, попробуйте снова")

ValueError: Index № invalid